In [1]:
import pygame
from multiprocessing import Process, Pipe

joysticks = []
pygame.init()
joysticks = [pygame.joystick.Joystick(x).get_name() for x in range(pygame.joystick.get_count())]
while joysticks == []:
    pygame.quit()
    pygame.init()
    joysticks = [pygame.joystick.Joystick(x).get_name() for x in range(pygame.joystick.get_count())]
print(joysticks)

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
['Logitech Extreme 3D']


In [2]:
if(type({}) == dict):
    print("chuj")

chuj
